# 共享变量的用途
在构建模型时，需要使用tf.Variable来撞见一个变量。例如：
___
```Python
#创建一个偏执的学习参数，在训练时，这个变量不断地更新。
biases = tf.Variable(tf.zeros([2],name="biases"))
```
___

在某种情况下，一个模型需要使用其他模型创建的变量，两个模型一起训练。比如，对抗网络中的生成器模型与判别器模型。如果使用tf。Variable,将会生成一个新的变量，而我们需要的是原来的那个biases变量。 这时就是通过引入**get_variable**方法。

## 使用get_variable获取变量
get_variable一般会配合**variable_scope**一起使用，以实现共享变量。variable_scope的意思是**变量作用域**。在某一作用域中的变量可以被设置成共享的方式，被其他网络模型使用。get_variable函数定义如下：
___
tf.get_variable(name, shape, initialzer)
___
TensorFlow中，使用get_variable属性为唯一标识，并不是定义的变量名称。使用时一般通过name属性定位到具体变量，并将其共享到其他模型中。

## get_variable和Variable的区别
1. Variable用法

In [3]:
import tensorflow as tf

var1 = tf.Variable(1.0, name='firstvar')
print("var1:",var1.name)
var1 = tf.Variable(2.0, name='firstvar')
print("var1:", var1.name)

var2 = tf.Variable(3.0)
print("var2:",var2.name)
var2 = tf.Variable(4.0)
print("var2:",var2.name)

with tf.Session() as sess:
    sess.run(tf.global_variables_initializer())
    print("var1=",var1.eval())
    print("var2=",var2.eval())

var1: firstvar_2:0
var1: firstvar_3:0
var2: Variable_2:0
var2: Variable_3:0
var1= 2.0
var2= 4.0


上面代码定义了两次var1，可以看到在内存中生成了两个var1（**因为它们的name不一样**）,对于图来讲后面的var1是生效的。
var2表明了：Variable定义时没有指定名字，系统会自动给加上一个名字Variable：0。

### get_variable用法演示

In [7]:
# 
tf.reset_default_graph()

get_var1 = tf.get_variable("firstvar",[1],initializer=tf.constant_initializer(0.3))
print("get_var1:",get_var1.name)

get_var1 = tf.get_variable("firstvar1",[1],initializer=tf.constant_initializer(0.4))
print("get_var1:",get_var1.name)

with tf.Session() as sess:
    sess.run(tf.global_variables_initializer())
    print("get_var1=", get_var1.eval())

get_var1: firstvar:0
get_var1: firstvar1:0
get_var1= [0.4]


### 在特定的作用域下获取变量
在作用域下，使用get_variable,以及嵌套variable_scope。
使用get_variable创建两个同样名字的变量是行不通的。
___
var1 = tf.get_variable("firstvar",shape=[2],dtype=tf.float32)
var2 = tf.get_variable("firstvar",shape=[2],dtype=tf.float32)
___

如果真的想要那么做，可以使用variable_scope将它们隔离开。

In [8]:
import tensorflow as tf 
with tf.variable_scope("test1",): #定义一个作用域test1
    var1 = tf.get_variable("firstvar",shape=[2],dtype=tf.float32)

with tf.variable_scope("test2"):
    var2 = tf.get_variable("firstvar",shape=[2],dtype=tf.float32)
    
print("var1:",var1.name)
print("var2:",var2.name)

var1: test1/firstvar:0
var2: test2/firstvar:0


生成的两个变量var1和var2是不同的，它们作用在不同的scope下，这就是scope的作用。
scope还支持嵌套，将上面代码中的地问个scope缩进一下：

In [10]:
# 重置图模型
tf.reset_default_graph()

with tf.variable_scope("test1",):
    var1 = tf.get_variable("firstvar",shape=[2],dtype=tf.float32)
    with tf.variable_scope("test2"):
        var2 = tf.get_variable("firstvar",shape=[2],dtype=tf.float32)

print("var1:",var1.name)
print("var2:",var2.name)

var1: test1/firstvar:0
var2: test1/test2/firstvar:0


### 共享变量功能的实现
> 使用作用域中的reuse参数来实现共享变量功能。variable_scope里面有个reuse=True属性，表示使用已经定义过的变量。这时get_varible将不会再创建新的变量，而是去图中get_variable所创建过的变量中找到name相同的变量。

In [11]:
with tf.variable_scope("test1",reuse=True):
    var3 = tf.get_variable("firstvar",shape=[2],dtype=tf.float32)
    with tf.variable_scope("test2"):
        var4 = tf.get_variable("firstvar",shape=[2],dtype=tf.float32)
        
print("var3:",var3.name)
print("var4:",var4.name)

var3: test1/firstvar:0
var4: test1/test2/firstvar:0


var1和var3的输出名字是一样的，var2和var4的名字也是一样的。这表明var1和var3共用了一个变量。
> 在实际应用中，可以吧var1和var2放到一个网络模型里去训练，把var3和var4放到另一个网络模型里去训练，而两个模型的训练结果都会作用于一个模型的学习参数上。

> tf.get_variable在创建变量时，回去检查图中是否已经创建过该变量。如果创建过并且本次调用时没有被设为共享方式，则会报错。加上tf.reset_default_graph()，将图里面的变量清空。

### 实例：初始化共享变量的作用域
variable_scope和get_variable都有着初始化的功能。在初始化时，**如果没有对当前变量初始化，则TensorFlow会默认使用作用域的初始化方法对其初始化，并且作用域的初始化方法也有继承功能。**

In [2]:
# 共享变量的作用域与初始化
import tensorflow as tf 

with tf.variable_scope("test1",initializer=tf.constant_initializer(0.4)):
    var1 = tf.get_variable("firstvar", shape=[2], dtype=tf.float32)
    
    with tf.variable_scope("test2"):
        var2 = tf.get_variable("firstvar",shape=[2],dtype=tf.float32)
        var3 = tf.get_variable("var3",shape=[2],initializer=tf.constant_initializer(0.3))
        
with tf.Session() as sess:
    sess.run(tf.global_variables_initializer())
    print("var1=",var1.eval()) # 作用域test1下的变量
    print("var2=",var2.eval()) # 作用域test2下的变量，继承test1初始化
    print("var3=",var3.eval()) # 作用域test2下的变量

var1= [0.4 0.4]
var2= [0.4 0.4]
var3= [0.3 0.3]


> 在多模型训练中，常常会使用variable_scope对模型间的张量进行区分。同时，统一为学习参数进行默认的初始化。在共享变量方面，还可以使用tf.AUTO_REUSE来为reuse属性赋值。tf.AUTO_REUSE可以实现第一次调用variable_scope时，传入的reuse值是false，再次调用variable_scope时，传入reuse的值就会变为True。

### 实例：演示作用域与操作符的受限范围
variable_scope还可以使用with varibale_scope("name") as xxxxscope 的方式定义作用域，当使用这种方式时，所定义的作用域变量xxxscope将不再受到外围的scope所限制。

In [13]:
import tensorflow as tf

tf.reset_default_graph()

with tf.variable_scope("scope1") as sp:
    var1 = tf.get_variable("v",[1])
   
    print("sp:", sp.name)  # 作用域名称
    print("var1:", var1.name) 

with tf.variable_scope("scope2"):
    var2 = tf.get_variable("v",[1])
    with tf.variable_scope(sp) as sp1:
        var3 = tf.get_variable("v3",[1])
        
        print('sp1',sp1.name)
        print('var2',var2.name)
        print("var3",var3.name)

sp: scope1
var1: scope1/v:0
sp1 scope1
var2 scope2/v:0
var3 scope1/v3:0
